In [58]:
import datetime
import hashlib
import json


In [59]:
class BlockChain:

    def __init__(self):
        self.chain = []
        self.create_block(proof = 1, previous_hash = '0') # genesis block

    def create_block(self, proof, previous_hash): 
        block = {
            'token Id' : len(self.chain) + 1, # number of block
            'artist name' : 'null',
            'picture name' : 'null',
            'wallet address' : 'null', #지갑 주소 (구매자 정보)
            'picture information' : 'Minted on {}. This work is unique.'.format(str(datetime.datetime.now())),
            'proof' : proof,    
            'previous_hash' : previous_hash 
        }
        self.chain.append(block) 
        return block

    def get_previous_block(self): # retrun previos_block
        return self.chain[-1]

    def proof_of_work(self, previous_proof): # 작업 증명
        new_proof = 1
        check_proof = False

        while check_proof is False: #work; 256길이의 hash가 '0000'을 포함할 때까지 반복 
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            
            if hash_operation.startswith('0000'): #proof
                check_proof = True
            else:
                new_proof +=1

        return new_proof #return work count

    def hash(self, block): # 블럭을 받아서 json형태로 dump하고, 인코딩하여 해시값을 얻은 후 반환
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return hashlib.sha256(encoded_block).hexdigest()


In [60]:
from flask import Flask, jsonify

app = Flask(__name__)
app.config['JSONIFY_PRETTYPRITNT_REGULAR'] = False
blockchain = BlockChain()


@app.route('/mine_block', methods = ['GET'])
def mine_block():
    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash =  blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)
    responses = {
        'message' : 'Congratulations, you jst mined a block!',
        **block
    }
    return jsonify(responses),200

@app.route('/get_chain', methods = ['GET'])
def get_chain():
    reponse = {
        'chain' : blockchain.chain
    }
    return jsonify(reponse), 200

app.run(host = '0.0.0.0', port = 6002)



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.139:6002/ (Press CTRL+C to quit)
192.168.1.139 - - [27/Jun/2022 22:10:18] "GET /mine_block HTTP/1.1" 200 -
192.168.1.139 - - [27/Jun/2022 22:10:18] "GET /mine_block HTTP/1.1" 200 -
192.168.1.139 - - [27/Jun/2022 22:10:20] "GET /get_chain HTTP/1.1" 200 -
192.168.1.139 - - [27/Jun/2022 22:10:20] "GET /get_chain HTTP/1.1" 200 -
